# Report project Basics of mobile robotics
Members: Mahdi Fourati, Yahya Hadi, Fatih Yilmaz, Alex Zanetta

## Introduction

This Jupyter Notebook serves as a final report for the project of Basics of mobile robotics. We will first go over our initial choices, then make a deep dive into the specific parts of the code.

### Initial choices
* Environment: we decided to use a simple blue floor with black obstacles and a red objective to have a strong contrast between them and the robot to make the vision easier. The obstacles can have any shape, which make the math heavier but more representative of the reality.
* Global navigation: the path from the starting point to the goal is computed using Dijkstra's algorithm. We decided to use Dijkstra's algorithm to allow us more flexibility in case we wanted to further complexify the problem and add some intermediary goals (which doesn't work with heuristics based algorithm such as A*). The points that the algorithm uses are the vertices of the obstacles approximated a bit more than half the robot's width away to avoid crashing into them while taking the uncertainty into account. 
* Pose estimation : the filter we used to estimate the pose and to control the robot is an extended Kalman filter. We chose it because all the prerequisite for its use where fullfilled and it is the de facto standard in navigation systems [[1]](#1).
* Motion control: we used a standard P-lead controller to make the robot follow the trajectory. As the trajectory is linear and the odometry reasonably good (the same speed on the two wheels will make the robot go straight with very little error), we don't need to make it more complex and use a PID controller or any othere more complex.
* Local navigation: this part makes the robot avoid crashing into obstacles if it senses something right in front of him. The shape of the obstacles is not relevant: the robot will move around it in the direction where it didn't sense anything. If all the sensors sense something, i.e. the robot is in a corner, it will do a 180°.




## Vision

### Getting the field of play

Code: vision.py/get_fop_coordinates(frame), vision.py/get_fop(frame, original_coordinates, new_coordinates)

To get the field of play (FoP), we begin by getting the four corners of the green rectangle on the raw video feed, and approximate it by a four corners polygon. In order to find the corners, we threshold the input image on blue and invert it. The operation of threshholding takes an image with a single colour channel (by opposition to the standard 3 colours RGB/BGR format), and sets all pixel below a threshold parameter to 0 (black) and all above to 255 (white):

$$
out(x,y)=
\begin{cases}
255 & \quad \text{when $src(x,y) > parameter$}\\ 
0 & \quad \text{otherwise}
\end{cases}
$$

We use a thresh parameter low enough to get the whole border but high enough to avoid getting the FoP. Finally, we invert the result. This yields the FoP as a white shape on a black background. This allows us to find the contours of the shape. They are all the boundary points of a white object on a black background. We make the computations more efficient by throwing away all unnecessary points (e.g. we don't need all points of a straight line, just the first and the last). This is the code doing this part:  

---
```r
_, thresholded_blue = cv.threshold(copy[:,:,0],127,255,cv.THRESH_BINARY_INV)
contours, _ = cv.findContours(thresholded, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
```
---


We chose to use a green floor to make the contrasts as large as possible with the other elements. Since the camera is not perfectly over the FoP, the raw video feed yields a tilted FoP with borders possibly containing unwanted elements. We must then cast this raw FoP onto a perfect rectangle more representative of the reality. To do this, we use a perspective transformation. We begin by approximating the contour by a four sided polygon using a Douglas-Peucker algorithm, which recursively divide the original contour until the variation between the newly computed contour and the original one is the one specified[[2]](#2). Once we have our 4 vertices, we compute the matrix casting them onto a perfect rectangle of width and height same as the maximum width and height of the polygon. We then apply this matrix to make a perspective transform of the raw video feed. This transformation is a combination of a similarity, a shear transformation, a scaling and an elation that can be formulated that way [[3]](#3): 

$$ 
out(x,y) =
\begin{bmatrix}
s \times cos(\theta) & -s \times sin(\theta) & t_{x} \\
s \times sin(\theta) & s \times cos(\theta) & t_{y} \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & k & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\lambda & 0 & 0 \\
0 & 1/\lambda & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
v_{1} & v_{2} & v
\end{bmatrix}
$$

These are the code snippets highlighting the main aformentioned points:

---
```r
# approximate the field of play to a polygon with 4 vertices
epsilon = 0.05 # 5% delta between contour and approximation
perimeter = cv.arcLength(c, True) # True means closed contour
approximation = cv.approxPolyDP(c, epsilon * perimeter, True)

# do the perspective transformation
matrix = cv.getPerspectiveTransform(original_coordinates, new_coordinates)
fop = cv.warpPerspective(frame, matrix, (int(new_coordinates[3][0]), int(new_coordinates[3][1])))
```
---

### Getting the robot starting position

Code: vision.py/get_robot_position(frame), vision.py/get_further_vertices(frame, robot_width, polygonal_obstacles)

It is quite easy to get the location of thymio with our set up. We simply look at the white zone on the FoP. To do this, we grayscale it to get the average BGR pixel intensity and threshholds on a high value. What is more complicated is getting the starting angle of the robot. This is done by looking for the curved part of the robot. We make two different polygonal approximations of the robot, both precise but with slightly different precisions. We then use a bitwise XOR to get the slight difference in curvature and compute the center of this difference. Once we have it, we compute the vector from the center of the robot to the center of the difference and get the complex argument of this vector, interpreting the y coordinate as the imaginary part. The argument is computed that way[[4]](#4):

$$
Arg(x,y)=
\begin{cases}
arctan(\dfrac{y}{x}) & \quad \text{when $x > 0$}\\ 
arctan(\dfrac{y}{x}) + \pi & \quad \text{when $x < 0 \: \& \: y \geq 0$}\\ 
arctan(\dfrac{y}{x}) - \pi & \quad \text{when $x < 0 \: \& \: y < 0$}\\ 
\dfrac{\pi}{2} & \quad \text{when $x = 0 \: \& \: y > 0$}\\ 
-\dfrac{\pi}{2} & \quad \text{when $x = 0 \: \& \: y < 0$}\\ 
undefined & \quad \text{when $x = 0 \: \& \: y = 0$}
\end{cases}
$$

### Getting the obstacles

Code: vision.py/get_obstacles(frame, robot_width)

The contours of the obstacles are obtained in a similar way than the FoP and the starting position of the robot, the math behind it has already been explained in said sections. What has not yet been explained is how we get the vertices placed futher away from the obstacles to avoid the robot crashing in them. We first use the angle bisector theorem to get the point opposite to the vertex of the obstacle. We then draw the line going through the vertex and the opposite point. We then make an intersection between this line and a circle of radius of the robot's width centered on the vertex. The intersection is computed by doing a bitwise AND operation:
$$
out(x,y)=
\begin{cases}
255 & \quad \text{when $thresh_{green}(x,y) \neq 0 \: \& \: thresh_{red}(x,y) \neq 0$}\\ 
0 & \quad \text{otherwise}
\end{cases}
$$

We get two intersections, one in the obstacle and one outside. We remove the one outside by making a bitwise substraction of a mask of the obstacles, and are left with the point we wanted.

### Getting the goal

Code: vision.py/get_objective(frame) 

The goal is obtained in a similar way to the robot and the obstacles: we threshold on a specific color, red in our case, and remove the robot from it using a bitwise XOR and AND.

## Global planning
 
In order to make our robot move from his starting position to the goal, we had to find the shortest path using all the vercities found by the vision to avoid any obstacle. To accomplish this feature, we are using a dijkstra algorithm. 

### Mapping

From the vision, we get an array of points representing each verticities of all obstacles, the starting position of the robot and the coordinates of the goal.

Since we have all the points in the map, we can name them and start looking for connexions between them. 
- Are they from the same obstacle ? 
- Are they connected ? 
- What distance separetes them ? 

To answer those questions, we used python dictionnaries and some useful insights from the web. 
For example, to check if two points are connected, we had to check if the straight line between them doesn't intersect with any line formed by two adjacent verticities of a same obstacle(instersect with an obstacle).
To figure out if two lines intersects, we used the concept of orientation[[5]](#5) that determines if 3 points are colinear, oriented clock-wise or counter clock_wise. 
Using this formula : 

$$
\text{orientation} = (x_3 - x_2)(y_2 - y_1) - (x_2 - x_1)(y_3 - y_2)
$$

If the orientation is above 0, then the the points are clock-wise oriented, if the orientation is belown 0, the points are counter clock-wise oriented and if the orientation is null, the three points are then colinear.

Now by calling _is_intersection_, we can determine if two lines intersect as shown by code snippet below :

```python
    ori1=orientation(point1,point2,point3)
    ori2=orientation(point1,point2,point4)
    ori3=orientation(point3,point4,point1)
    ori4=orientation(point3,point4,point2)
```
and 

```python
    if ori1 != ori2 and ori3 != ori4:
        return True # they intersect
```
For the distances, we used for the entirety of the global_planning the euclidian distance. 
This is the formula that we used: 
$$ 
d(p_1, p_2) = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}
$$

Now that we have dictionaries of connected points and the distances between them, we can go on and start finding the shortest path to move the robot from its starting position to the goal. 




### Dijkstra algorithm

We chose to use this algorithm for our project because it is fairly simple to implement in addition to being very flexible (e.g not needing a grid) and the documentation on it is abundant compared to other algorithms.

In order to implement this famous algorithm, we took as template the algorithm found on this webpage [[6]](#6).
This is also what inspired us the use of dictionnaries in the first place as they are a very convinient in order to create graphs.

Basically, the algorithm works by getting the graph (in our case a dictionnary of adgencency + dictionnary of the coordinates of points in order to calculate the distance between them). Having done that, we compute the best path for the robot by checking every node starting with the robot, mesuring and comparing distances between node and keeping the nodes that produces the shortest path. Finally, by calling the function _global_plan_ we get the shortest path staring from the robot, avoiding obstacles until it hits the goal. 

You can find below a code snippet that showcases how the dijkstra algorithm works : 

```python
while unvisited:
    current = unvisited[0]
    for node in unvisited:
        if shortest_dist[node] < shortest_dist[current]:
            current = node
    neighbors = adjacency_dict[current]
    for i in neighbors:
        try_value = shortest_dist[current] + get_dist(distance_dict, current, i)
        if try_value < shortest_dist[i]:
            shortest_dist[i] = try_value
            previous_nodes[i] = current
    unvisited.remove(current)
```

Here,
- _unvisited_ is a list of all the point/nodes that we didn't explore yet in terms of establishing deistances. 
- _shortest_dist_ is a dictionnary containing the best known cost for getting to each node.
- _previous_nodes_ is a dictionnary containing the best known path for each node.

### Example
<center>
    <img src="Global mapping.jpg" alt="Global Mapping">
<center>
Figure 1: Map with path and points displayed
<br>
<br>

<div style="text-align: left;">
We can see here that first of all every point has been named and put in dictionnaries (points_named,adgecency_dict etc) to be used later for Dijkstra. We can also see the path represented by the blue line is this case going from the robot 'R', into 'P3', then 'P12', 'P9' and finally reaching the goal 'G'.
</div>

## Motion control

### Controlling the robot using a PI

Code: controlling_thymio.py/PI_controller(error, kp, ki)

We use two PI controllers, one to correct the angle of the robot and one to correct how far it is from the optimal trajectory. All parameters have been fine tuned by experimenting and keeping the best of them

---
```r
system_input = kp * error[-1] + ki * np.sum(error)
```
---

### Computing the linear error

Code: controlling_thymio.py/get_linear_error(start_point, end_point, robot_center)

To compute the distance between the robot R and the optimal trajectory between P1 and P2, we use the cross product:

$$ 
d = \frac{\|(P2-P1) \times (P1-R)\|}{\|(P1-R)\|}    
$$

Geometrically, we get the area of the parallelogram generated by the sides P1-P2 and P1-R and divide by the base to get the height. We then use the same method than in the mapping to get on which side of the trajectory we are.

### Computing the angular error


### Local avoidance 

During the journey, our robot is not immune to encountering unexpected obstacles. With the help of horizontal proximity sensors, we can avoid these obstacles. Our robot has 7 horizontal proximity sensors, with 5 at the front and 2 at the rear. These sensors are infrared and operate at a frequency of 10 Hz.  

We have a primary function, check_local_nav, which checks if a local obstacle is detected and switches to local navigation if necessary. If we enter local navigation, we call the local_avoidance function. This function sets the motor speeds to avoid obstacles and returns the latest speed values to update the robot instance and provide the necessary information for motion control or the Kalman filter.  

The side_obst function is used to avoid obstacles located on the robot's sides. This is done by examining the values of the horizontal sensors located at the extremities. We use the sensors at the edges because this function is meant to avoid obstacles located at the extremities. Once the sensor values exceed a detection threshold OBST_THR, we update the motor speeds and return them. The value of the threshold is 2500. We chose this value because, after several tests, we observed that it corresponds to a distance of approximately 5.25 cm, and the robot performed well with this setting. We consider this value appropriate as it enables the robot to effectively avoid a nearby obstacle at a close distance while being large enough to prevent any collisions.

<br>
<center>
    <img src="side_obst_image.png" width="500">
<center>
    Figure 2: Side obstacle scheme
<br>
<br>
    
<div style="text-align: left;">
The <i>front_obst</i> function works similarly but includes additional variables, <i>close_left</i>  and <i>close_right</i> , which are the averages of the left and right sensor values. Using average variables for the left and right sensors offers several advantages. It simplifies the code by reducing repetition, making it more readable and easier to understand. The average also helps smooth out variations or noise from the sensors, providing a better estimation of obstacle proximity. By directly comparing two averages (left and right), decisions become more balanced and easier to manage. Finally, this approach makes the code more modular and easier to maintain, as any changes to the sensors can be centralized in the average calculation. Here, we use the five horizontal front sensors because the center sensor detects a frontal obstacle, while the other four determine whether the obstacle is more to the left or the right using the average variables <i>close_left</i>  and <i>close_right</i> . The detection threshold is the same as for the side obstacles, 2500 (approximately 5.25 cm), for the same reasons.
</div>

<br>
<center>
    <img src="front_obst_image.png" width="500">
<center>
    Figure 3: Front obstacle scheme

<br>
<br>

<div style="text-align: left;">
The <i>update_local_state</i> function updates the local state. It is the key function for the proper functioning of local navigation. We check the value of each front horizontal sensor and set the corresponding index in a list to 1. This way, we have access to a very simple list that indicates whether a sensor has detected something. We can then update the state directly by analyzing the list composed of 0s and 1s.
</div>

<br>

<div style="text-align: left;">
The speeds were chosen based on the type of obstacle encountered. For a side obstacle, since the obstacle is not directly in front of the robot's trajectory, we can accelerate one wheel relative to the other to avoid it while continuing to move forward along the trajectory. This is why, in the <i>side_obst</i>  function, we use two speed constants: <i>SPEEDROT1</i> equal to 450 and <i>SPEEDROT2</i> equal to 75. For example, if an obstacle is on the left, we set the left wheel to <i>SPEEDROT1</i> and the right wheel to <i>SPEEDROT2</i>.

<br>

For a frontal obstacle, we prefer to stop the robot from advancing along its trajectory. We determine on which side the obstacle is located. For instance, if the obstacle is on the right, we set the right wheel speed to <i>SPEEDROT1</i>  and the left wheel speed to zero. This way, the robot will turn until it no longer detects the obstacle as frontal, but instead as a side obstacle. It will then avoid the obstacle as described earlier.
</div>

<br>
<center>
    <img src="combination_obst_image.png" width="500">
<center>
    Figure 4: Combination of front and side obstacle scheme

## Kalman filter


To estimate the pose of our Thymio robot from noisy measurements like a webcam we use the Kalman filter. Since our Thymio is a simple differential drive robot, we can only use the x, y coordinates and the angle to estimate its position.\
Since we assume that the robot moves in a pseudo-linear fashion, we only use the regular Kalman filter and not the extended version. 
The state of the system is represented by a vector $x$ and the measurements by a vector $z$. The Kalman filter estimates the state of the system by computing the mean and the covariance of the state vector.\
Our motion model is given by the following equations:



### Motion model


$$
\mathbf{x}_{k+1} =
\begin{bmatrix}
x_{k+1} \\
y_{k+1} \\
\theta_{k+1} \\
\end{bmatrix}
=
\begin{bmatrix}
x_k + \cos(\theta_k) \cdot v   \\
y_k + \sin(\theta_k) \cdot v \ \ \\
\theta_k + \omega  \\

\end{bmatrix}

$$

where $v$ is the linear velocity = ($w_{L, k}$ + $w_{R, k}$)/2 , $\omega$ the angular velocity = ($w_{L, k}$ - $w_{R, k}$)/wheel_base , $w_{L, k}$ and $w_{R, k}$ the left and right wheel velocities at time $k$ and $\Delta t$ the time step.

### Measurement Model

The utility of the Kalman filter is to estimate the state of the system even without camera. This way we have two cases : when the camera is available the measurement model is given by the following equations:

$$

H =
\begin{bmatrix}
1 & 0 & 0  \\
0 & 1 & 0  \\
0 & 0 & 1  \\

\end{bmatrix}
$$
$$
z_{k} =
\begin{bmatrix}
x_{\text{cam}} \\
y_{\text{cam}} \\
\theta_{\text{cam}} \\
\end{bmatrix}


$$

For the case when the camera is obstructed and we can't get the measurements, we only predict the state of the system using the motion model and we don't update the state using the measurements since all our measurements come from the camera.

### Covariance matrices

To estimate the measurement and process noise, we perform some experiments to get the standard deviation of the noise. We then use these values to compute the covariance matrices $Q$ and $R$.\
To estimate the measurement noise, fsor the coordinate x and y of the robot, we place the robot in a static position and use the camera to retrieve the coordinates. Then we compute the standard deviation on the x and y direction.
We recorded a of around 5 pixels in both direction, 
For the angle we performed a similar experience where we gave the robot a target speed of 100 and -100 so that it turns on itself for a period a time and recorded the variance in the position it stopped at. This gave us a variance of about 1 degree which is 0.017 radians.

$$
R
=
\begin{bmatrix}
\sigma_{x}^2 & 0 & 0 \\
0 & \sigma_{y}^2 & 0  \\
0 & 0 & \sigma_{\theta}^2  \\

\end{bmatrix}
=
\begin{bmatrix}
5 & 0 & 0  \\
0 & 5 & 0   \\
0 & 0 & 0.017  \\

\end{bmatrix}
$$

On the other hand, the process noise matric $Q$ represents the noise in our model. Since our model doesn't account for plenty factors for example, we don't take into consideration the wheel slippage (due to the material the robot is on), noise in the motors etc. We estimate it by trial and error. We found that a value of 1.0 for the diagonal of the matrix gives us good results.

$$
Q

=
\begin{bmatrix}
1 & 0 & 0  \\
0 & 1 & 0   \\
0 & 0 & 1  \\

\end{bmatrix}
$$

### Implementation
For the implementation of the Kalman filter, we were inspired by the implementation from the course in week 8 and also from the wikipedia article [[1]](#1).\
We created a class KalmanFilter that implements multiple functions like predict that predicts the a priori estimate of the state using our motion model, the update that updates the state using the measurements, the innovation and the Kalman gain.\
In our convert input function, we use the wheels speeds of the robot to compute the linear velocity then compute our change in the state vector that we use to predict the a priori estimate of the state.\
For the delta angle, we experimented with multiple speeds to make the robot turn and recorded the angle it turned at. We then used this data to do a linear approximation of the angle

In [ ]:
import numpy as np

DISTANCE_WHEEL = 90 # distance between the wheels in mm
CAMERA_FPS = 30 # fps of the video feed
DISTANCE_MM_S = 1 # distance done in mm/s per 1 PMW

def convert_input(robot):
    """
    Convert B*u, the effect of the system input
    Input:
        robot: an instance of the class Robot
    """
    # convert the speed values of the wheel to pixel
    linear_velocity = (robot.lspeed + robot.rspeed)/2 * DISTANCE_MM_S / CAMERA_FPS * robot.robot_width / DISTANCE_WHEEL
   
    # linear approximation of the angle based on experimentation 
    delta_angle = (0.1667 * (robot.rspeed - robot.lspeed) + 8.33) * np.pi / 180

    angle = float(robot.alpha)
    delta_x = np.cos(angle) * linear_velocity
    delta_y = np.sin(angle) * linear_velocity
    
    return [delta_x, delta_y, delta_angle]
class KalmanFilter:
    
    def __init__(self, x0):
        self.F = np.array([[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]) # state transition model
        self.H = np.array([[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]) # observation model
        self.Q = np.array([[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]) # process noise covariance
        self.R = np.array([[5.0, 0, 0], [0, 5.0, 0], [0, 0, 0.017]]) # measurement noise covariance
        self.P = np.array([[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]) # initial error covariance  
        self.x = x0 # initial state, shape [x,y,alpha]

    def predict(self, converted_input):
        """
        Use the Kalman filter to make a prediction of the future state
        Input:
            - converted_input: B*u, the effect of the system input
        """
        self.x = self.F @ self.x + converted_input
        self.P = self.F @ self.P @ self.F.transpose() + self.Q

        # to keep the angle bewteen )-pi;pi]
        if self.x[2][0] > np.pi:
             self.x[2][0] -= 2*np.pi
        elif self.x[2][0] < -np.pi:
            self.x[2][0] += 2*np.pi
 
    def update(self, z):
        """
        Update the Kalman filter using observations to make better estimations
        Input:
            - z: observation of the robot, shape [x,y,alpha]
        """
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.transpose() + self.R   
        K = self.P @ self.H.transpose() @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P

        # to keep the angle bewteen )-pi;pi]
        if self.x[2][0] > np.pi:
             self.x[2][0] -= 2*np.pi
        elif self.x[2][0] < -np.pi:
            self.x[2][0] += 2*np.pi

## Sources

[1]: <a id="1">Wikipedia, “Extended Kalman filter” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Extended_Kalman_filter</a>

[2]: <a id="2">Wikipedia, “Ramer–Douglas–Peucker algorithm” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Ramer%E2%80%93Douglas%E2%80%93Peucker_algorithm</a>

[3]: <a id="3">Medium, “Part II: Projective Transformations in 2D” [Last access 26 November 2024]. Available: https://medium.com/@unifyai/part-ii-projective-transformations-in-2d-2e99ac9c7e9f</a>

[4]: <a id="4">Wikipedia, “Argument (complex analysis)” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Argument_(complex_analysis)</a>

[5]:  <a id="5">Geeksforgeeks, “Orientation of 3 ordered points” [Last access 19 November 2024]. Available: https://www.geeksforgeeks.org/orientation-3-ordered-points/</a>

[6]: <a id="6">Udacity, “Implementing Dijkstra’s Algorithm in Python” [Last access 19 November 2024]. Available: https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html</a>


